In [5]:
import os
from ossapi import Ossapi
import sqlite3
from time import strftime, localtime
import time
from concurrent.futures import ThreadPoolExecutor
import sys
from numpy import array_split
import pickle
from datetime import datetime
sys.path.insert(0, "../") # Required for database in different directory. 

OSU_CLIENT_ID = os.environ.get('OSU_CLIENT_ID')
OSU_CLIENT_SECRET = os.environ.get('OSU_CLIENT_SECRET')

api = Ossapi(OSU_CLIENT_ID, OSU_CLIENT_SECRET)

top_or_recent = "top"
mode = "std"


In [6]:
def scrape_beatmaps(beatmap_ids):
    global num_scraped

    conn = sqlite3.connect("../UserScores.db")
    cursor = conn.cursor()

    queue = []
    for beatmap_id in beatmap_ids:
        try:
            beatmap = api.beatmap(beatmap_id)
        except Exception as e:
            print(e)
            print("failed to get beatmap", beatmap_id)

        accuracy = getattr(beatmap, "accuracy", None)
        ar = getattr(beatmap, "ar", None)
        beatmap_id = beatmap_id  # no shit
        beatmapset_id = getattr(beatmap, "beatmapset_id", None)
        bpm = getattr(beatmap, "bpm", None)
        cs = getattr(beatmap, "cs", None)
        difficulty_rating = getattr(beatmap, "difficulty_rating", None)
        drain = getattr(beatmap, "drain", None)
        max_combo = getattr(beatmap, "max_combo", None)
        owner_user_id = getattr(beatmap, "owner_user_id", None)
        total_length = getattr(beatmap, "total_length", None)
        url = getattr(beatmap, "url", None)
        version = getattr(beatmap, 'versoin', None)
        
        queue.append(
            (
                accuracy,
                ar,
                beatmap_id,
                beatmapset_id,
                bpm,
                cs,
                difficulty_rating,
                drain,
                max_combo,
                owner_user_id,
                total_length,
                url,
                version
            )
        )

        if len(queue) > 100:
            cursor.executemany(
                """
                INSERT INTO beatmaps_std (
                accuracy, ar, beatmap_id, beatmapset_id, bpm, cs,
                difficulty_rating, drain, max_combo, owner_user_id,
                total_length, url, version
                ) 
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """,
                queue,
            )
        queue.clear()

In [7]:
conn = sqlite3.connect('../data/UserScores.db')
cursor = conn.cursor()

mode = 'std'

top_beatmap_ids = cursor.execute(f"SELECT DISTINCT beatmap_id FROM top_scores_{mode}").fetchall()
recent_beatmap_ids = set()
# recent_beatmap_ids = cursor.execute(f'SELECT DISTINCT beatmap_id FROM recent_scores_{mode}').fetchall()

completed_beatmap_ids = cursor.execute(f'SELECT DISTINCT beatmap_id from beatmaps_{mode}').fetchall()

remaining_beatmap_ids = list(set(top_beatmap_ids) | set(recent_beatmap_ids) - set(completed_beatmap_ids))

print(f"Remaining beatmap ids to scrape: {len(remaining_beatmap_ids)}")

num_partitions = 4
partitioned_beatmap_ids = array_split(remaining_beatmap_ids, num_partitions)

for beatmap_ids in partitioned_beatmap_ids:
    print(len(beatmap_ids))

with ThreadPoolExecutor(max_workers=num_partitions) as executor:
    for beatmap_ids in partitioned_beatmap_ids:
        executor.submit(scrape_beatmaps, beatmap_ids)

Remaining beatmap ids to scrape: 106110
21222
21222
21222
21222
21222
api returned an error of `Too Many Attempts.` for a request to https://osu.ppy.sh/api/v2/beatmaps/lookup?id=4115309
failed to get beatmap [4115309]
api returned an error of `Too Many Attempts.` for a request to https://osu.ppy.sh/api/v2/beatmaps/lookup?id=528857
failed to get beatmap [528857]
api returned an error of `Too Many Attempts.` for a request to https://osu.ppy.sh/api/v2/beatmaps/lookup?id=251753
failed to get beatmap [251753]
api returned an error of `Too Many Attempts.` for a request to https://osu.ppy.sh/api/v2/beatmaps/lookup?id=172610
failed to get beatmap [172610]
api returned an error of `Too Many Attempts.` for a request to https://osu.ppy.sh/api/v2/beatmaps/lookup?id=68878
failed to get beatmap [68878]
